In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df = spark.read.csv("/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv", header="true", inferSchema="true")
display(df.limit(10))

In [0]:
df_clean = df.toDF(*[c.replace(" ","_").lower() for c in df.columns])

In [0]:
display(df_clean.limit(10))

In [0]:
df.createOrReplaceTempView("fire_view")
# display(spark.table("fire_table"))

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS fire_db

In [0]:
spark.sql("DROP TABLE IF EXISTS fire_db.fire_table")

In [0]:
df_clean.write.mode("overwrite").saveAsTable("fire_db.fire_table")

In [0]:
%sql
SELECT * FROM fire_db.fire_table limit 1;

In [0]:
%sql
--Q1 Count of Distinct Calls made to the dept
SELECT COUNT(DISTINCT calltype)  AS
TotalCallTypes
from fire_db.fire_table

In [0]:
%sql
--Q2 Distinct Types of calls made to the fire dept
SELECT DISTINCT(calltype) as CallType
FROM fire_db.fire_table;


In [0]:
%sql
SELECT * from fire_db.fire_table limit 10;

In [0]:
%sql
-- Q3 To find all call numbers where delay was greater than 5 minutes
SELECT call_number, round(delay,2) as r_delay FROM fire_db.fire_table
WHERE delay > 5
ORDER BY r_delay DESC;

In [0]:
%sql
-- Find Most Common Call Types
SELECT calltype, COUNT(calltype) as count_of_call_type  from fire_db.fire_table
GROUP BY calltype
ORDER BY count_of_call_type DESC
LIMIT 5

In [0]:
%sql
-- What zip code accounted for most calls?
SELECT calltype, zipcode_of_incident, count(zipcode_of_incident) as count_of_zipcode
FROM fire_db.fire_table
GROUP BY zipcode_of_incident, calltype
ORDER BY count_of_zipcode DESC


In [0]:
%sql
-- What SF neighbhorhoods are in zip code 94102 and 94103

SELECT DISTINCT neighborhood, zipcode_of_incident
FROM fire_db.fire_table where zipcode_of_incident == 94102 OR zipcode_of_incident == 94103


--select * from fire_db.fire_table limit 10;

In [0]:
%sql
-- What was the sum of all call alarms, average, min and max of the call response time?
-- select * from fire_db.fire_table limit 10;

SELECT sum(numalarms) as total_no_of_calls, round(avg(delay),2) as avg_delay, round(min(delay),2) as min_delay, round(max(delay),2) as max_delay FROM fire_db.fire_table

In [0]:
%sql
-- How amny distinct years fo data is present in the dataset
SELECT DISTINCT year(to_date(call_date, "MM/dd/yyyy")) as year_num
FROM fire_db.fire_table
ORDER BY year_num ASC


In [0]:
%sql
-- What week of the year 2018 had the most fire calls
SELECT weekofyear(to_date(call_date, "MM/dd/yyyy")) as year_week, count(*) as count
FROM fire_db.fire_table
WHERE year(to_date(call_date,"MM/dd/yyyy")) == 2018
GROUP BY year_week
ORDER BY count DESC

In [0]:
%sql
-- What nighborhoods in SF had the worst response time in 2018

SELECT neighborhood, round(delay,2) as delay from fire_db.fire_table
WHERE year(to_date(call_date,"MM/dd/yyyy")) == 2018
ORDER BY delay DESC